# Implémentation d'un Chatbot documentaire (RAG)

## PARTIE 1 : Récupération de la base de données

Les rapports du GIEC (**Groupe intergouvernemental d’experts sur l’évolution du climat**) ou IPCC en anglais, fournissent un état des lieux régulier des connaissances les plus avancées sur le changement climatique, ses causes, ses impacts et les mesures possibles pour l’atténuer et s’y adapter.

La synthèse du sixième rapport d’évaluation du GIEC a été publiée le lundi 20 mars 2023. Fruit d’une collaboration internationale, ce nouveau rapport synthétise les connaissances scientifiques acquises entre 2015 et 2021. D'autres rapports ont été publiés entre temps sur des sujets spécifiques.

Nous nous intéressons à quatre de ces documents:

*   Sixth Assessment Report
*   The Ocean and Cryosphere in a Changing Climate
*   Climate Change and Land
*   Global Warming of 1.5°C




### 1. Récupération des documents

In [1]:
# Créez un dossier 'RAG_IPCC' dans les fichiers de votre session colab

import os

folder_path = "/content/RAG_IPCC"
os.makedirs(folder_path)

In [9]:
# Téléchargez les 4 fichiers dans ce dossier

url_6th_report = "https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_FullVolume.pdf"
url_ocean = "https://www.ipcc.ch/site/assets/uploads/sites/3/2022/03/02_SROCC_TS_FINAL.pdf"
url_land = 'https://www.ipcc.ch/site/assets/uploads/sites/4/2022/11/SRCCL_Technical-Summary.pdf'
url_warming = 'https://www.ipcc.ch/site/assets/uploads/sites/2/2022/06/SPM_version_report_LR.pdf'

for url in [url_6th_report, url_ocean, url_land, url_warming]:
  !wget -P /content/RAG_IPCC {url}

--2024-05-31 19:50:17--  https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_FullVolume.pdf
Resolving www.ipcc.ch (www.ipcc.ch)... 104.20.255.3, 104.20.254.3, 172.67.16.107, ...
Connecting to www.ipcc.ch (www.ipcc.ch)|104.20.255.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4913496 (4.7M) [application/pdf]
Saving to: ‘/content/RAG_IPCC/IPCC_AR6_SYR_FullVolume.pdf’

IPCC_AR6_SYR_FullVo 100%[===================>]   4.69M  --.-KB/s    in 0.1s    

2024-05-31 19:50:18 (43.8 MB/s) - ‘/content/RAG_IPCC/IPCC_AR6_SYR_FullVolume.pdf’ saved [4913496/4913496]

--2024-05-31 19:50:18--  https://www.ipcc.ch/site/assets/uploads/sites/3/2022/03/02_SROCC_TS_FINAL.pdf
Resolving www.ipcc.ch (www.ipcc.ch)... 104.20.255.3, 104.20.254.3, 172.67.16.107, ...
Connecting to www.ipcc.ch (www.ipcc.ch)|104.20.255.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4366638 (4.2M) [application/pdf]
Saving to: ‘/content/RAG_IPCC/02_SROCC_TS_FINAL.pdf’

### 2. Extraction du contenu textuel

In [3]:
# Choisissez une méthode d'extraction du contenu du pdf page à page

!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 30.2 MB/s eta 0:00:00


In [11]:
import pymupdf

list_pdfs = os.listdir("RAG_IPCC")
extracted_text = []
for pdf in list_pdfs:
  print(f"*** PROCESSING FILE : {pdf} ***")
  file_path = os.path.join(folder_path, pdf)
  doc = pymupdf.open(file_path)
  number_of_pages = doc.page_count
  print(f"Number of pages : {number_of_pages}")
  for n, page in enumerate(doc):
    page_text = page.get_text()
    extracted_text.append({"document": pdf, "page": n, "content": page_text})


*** PROCESSING FILE : SRCCL_Technical-Summary.pdf ***
Number of pages : 40
*** PROCESSING FILE : IPCC_AR6_SYR_FullVolume.pdf ***
Number of pages : 186
*** PROCESSING FILE : 02_SROCC_TS_FINAL.pdf ***
Number of pages : 34
*** PROCESSING FILE : SPM_version_report_LR.pdf ***
Number of pages : 24


### 3. Création des chunks

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Implémentez une fonction de splitting par nombre de mots

def splitting_by_numer_of_words(text, chunk_size):
  """
  Découpe un texte en chunks de taille donnée (nombre de caractères).

  Args:
    text (str): Le texte à splitter.
    chunk_size (int): La taille souhaitée des chunks (nombre de mots).

  Returns:
    list: Une liste de chunks de texte.
  """
  chunks = []
  for phrase in text.split('\n'):
    words = phrase.split()
    for i in range(0, len(words), chunk_size):
      chunks.append(' '.join(words[i:i + chunk_size]))
  return chunks

# Implémentez une fonction de splitting par phrase

def splitting_by_sentences(text):
  """
  Découpe un texte en chunks par phrases.

  Args:
    text (str): Le texte à découper.

  Returns:
    list: Une liste de chunks de texte (phrases).
  """
  sentences = nltk.sent_tokenize(text)
  return sentences

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
# Implémentez une fonction de splitting intelligente avec différents paramètres
 #(nombre maximal de mots, caractère de fin de chunks etc.)

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
# Créez vos chunks avec la fonction de splitting qui semble la plus pertinente
# ATTENTION : on veut garder un maximum de metadonnées dans la base (titre, page etc.)

chunks = []
for page_content in extracted_text:
  chunks_list = text_splitter.split_text(page_content['content'])
  # chunks_list = splitting_by_numer_of_words(page_content['content'])
  # chunks_list = splitting_by_sentences(page_content['content'])
  for chunk in chunks_list:
    chunks.append({"document": page_content['document'],
                   "page": page_content['page'],
                   "content": chunk})

chunks

[{'document': 'IPCC_AR6_SYR_FullVolume.pdf',
  'page': 0,
  'content': '1\nCLIMATE CHANGE 2023\nSynthesis Report\nA Report of the Intergovernmental Panel on Climate Change'},
 {'document': 'IPCC_AR6_SYR_FullVolume.pdf',
  'page': 2,
  'content': 'CLIMATE CHANGE 2023\nSynthesis Report\nHoesung Lee (Chair), Katherine Calvin (USA), Dipak Dasgupta (India/USA), Gerhard Krinner (France/Germany), Aditi Mukherji \n(India), Peter Thorne (Ireland/United Kingdom),\xa0Christopher Trisos (South Africa), José Romero (Switzerland), Paulina Aldunce \n(Chile), Ko Barrett (USA), Gabriel Blanco (Argentina), William W. L. Cheung (Canada), Sarah L. Connors (France/United Kingdom),'},
 {'document': 'IPCC_AR6_SYR_FullVolume.pdf',
  'page': 2,
  'content': 'Fatima Denton (The Gambia), Aïda Diongue-Niang (Senegal), David Dodman (Jamaica/United Kingdom/Netherlands), Matthias \nGarschagen (Germany), Oliver Geden (Germany), Bronwyn Hayward (New Zealand), Christopher Jones (United Kingdom), Frank \nJotzo (Australi

### 4. Nettoyage de la base documentaire

In [ ]:
# Créez une fonction pour nettoyer le contenu de chaque chunk

special_chars = [" ", '-', '&', '(', ')', '_', ';', '†', '+', '–', "'", '!', '[', ']', '’', '́', '̀', '\u2009', '\u200b', '\u202f', '©', '£', '§', '°', '@', '€', '$', '\xa0', '~','\n','�']

def remove_char(text, char):
    """Remove each specific character from the text for each character in the chars list."""
    return text.replace(char, ' ')

def remove_chars(text, chars):
    """ Apply remove_char() function to text """
    for char in chars:
        text = remove_char(text, char)
    return text

def remove_multiple_white_spaces(text):
    """Remove multiple spaces."""
    text = re.sub(" +", " ", text)
    return text

def clean_text(text, special_chars=special_chars):
    """Generate a text without chars expect points and comma and multiple white spaces."""
    text = remove_chars(text, special_chars)
    text = remove_multiple_white_spaces(text)
    return text



In [ ]:
# Créez différentes fonction pour retirer les chunks sans intérêt


def remove_short_chunks(chunks, min_length=5):
    return [chunk for chunk in chunks if len(chunk["content"].split()) >= min_length]

import re

def contains_mainly_digits(text, threshold=0.5):
    """
    Checks if a text string contains a high percentage of digits compared to letters.

    Args:
        text (str): The input text to analyze.
        threshold (float, optional): The threshold value for the proportion of digits to letters.
            Defaults to 0.5.

    Returns:
        bool: True if the proportion of digits in the text exceeds the threshold, False otherwise.
    """
    if not text:
        return False
    letters_count = 0
    nbs_count = 0
    for char in text:
        if char.isalpha():
            letters_count += 1
        elif char.isdigit():
            nbs_count += 1
    if letters_count + nbs_count > 0:
        digits_pct = (nbs_count / (letters_count + nbs_count))
    else:
        return True
    return digits_pct > threshold

def remove_mostly_digits_chunks(chunks, threshold=0.5):
  return [chunk for chunk in chunks if not contains_mainly_digits(chunk['content'])]

# (Eventuellement utiliser le layout du pdf avec pdfminer pour retirer les en-têtes et les pieds de page)


### BONUS : Augmentation des métadonnées

Avoir un maximum d'informations sur chaque chunk (titre du document, page, nom du chapitre, description du document, date ...) est toujours intéressant : comme informations complémentaires pour l'utilisateur mais aussi potentiellement pour affiner la recherche en elle-même.

In [ ]:
# Implémentez des fonctions permettant d'ajouter des métadonnées

# Utilisez les différentes polices et tailles de police pour retrouver le sous-titre antérieur le plus proche (bibliothèque pdfminer)
